## Esercizio 49 - Oil blending

I problemi di miscelazione sono una tipica applicazione industriale della Linear Programming (LP).

Una compagnia petrolifera produce $n$ tipi di combustibile $F = \{1, \dots, n\}$. Ogni tipologia di carburante viene prodotta miscelando $m$ tipi di oli grezzi $O=\{1, \dots, m\}$ che devono essere acquistati. La società deve decidere la quantità di petrolio greggio da acquistare al fine di massimizzare i suoi profitti nel rispetto delle capacità di lavorazione e dei livelli di qualità, nonché di soddisfare la domanda dei clienti.

Il problema della miscelazione dell'olio consiste nel calcolare diverse miscele di combustibile secondo specifici criteri di qualità.

Ogni combustibile $f \in F$ viene venduto al prezzo $p^F_f$ dollari e deve soddisfare alcuni criteri di qualità per quanto riguarda il contenuto di piombo (livello massimo $L^{lead}_f$) e il numero di ottani (livello minimo $L^{oct}_f$), limitando così le possibili miscele. Ogni olio grezzo $o \in O$ ha un costo $p^O_o$, un livello di piombo $l^{lead}_o$ e di ottani $l^{oct}_o$.

L'azienda deve anche soddisfare la domanda $d_f$ dei suoi clienti in termini di numero di barili al giorno per ciascun combustibile $f \in F$.

La società può acquistare $C_o$ barili di ogni tipo di petrolio greggio $o \in O$ al giorno e può processare al massimo $M$ barili al giorno.

Inoltre, la società ha la possibilità di pubblicizzare un combustibile, nel qual caso la domanda per questo tipo di carburante aumenta di $R$ barili per ogni dollaro speso.

Infine, costa $P$ dollari per trasformare un barile di petrolio in un barile di combustibile.

Vogliamo decidere la quantità $b_{of}$ di olio $o \in O$ da utilizzare per produrre il carburante $f \in F$. Inoltre vogliamo decidere quanto spendere, $a_f$, in pubblicità per ciascun combustibile $f \in F$.


\begin{align}
\max \quad & \sum\limits_{o \in O\\f \in F} p^F_f b_{of} - \sum\limits_{o \in O\\f \in F} p^O_o b_{of} - P \sum\limits_{o \in O\\f \in F} b_{of} - \sum\limits_{f \in F} a_f &  \\
s.t.\quad & \sum\limits_{o \in O} b_{of} = d_f + R \ a_f& f \in F\\ 
\quad & \sum\limits_{f \in F} b_{of} \leq C_o & o \in O\\
\quad & \sum\limits_{o \in O} (l^{oct}_o - L^{oct}_f) \ b_{of} \geq 0 & f \in F \\
\quad & \sum\limits_{o \in O} (l^{lead}_o - L^{lead}_f) \ b_{of} \leq 0 & f \in F \\
\quad & \sum\limits_{o \in O\\f \in F} b_{of} \leq M & \\
\quad & b_{of} \geq 0 & o \in O, f \in F\\
\quad & a_f \geq 0 & f \in F
\end{align}

* TEST: oils.json
> Objective: 287750<br>
> Ricavo totale: 830000<br>
> Totale costo petroli: 487500<br>
> Totale costo produzione: 54000<br>
> Totale costo pubblicità: 750<br>
> composizione super: 2088.89 di crude1, 777.78 di crude2, 133.33 di crude3<br>
> composizione regular: 2111.11 di crude1, 4222.22 di crude2, 3166.67 di crude3<br>
> composizione diesel: 800.0 di crude1, 0 di crude2, 200.0 di crude3<br>
> costo pubblicità su super: 0<br>
> costo pubblicità su regular: 750<br>
> costo pubblicità su diesel: 0<br>

In [ ]:
# ALERT: execute this cell to prepare input data!
import requests
def download(link, nomeFile=None):
    if nomeFile == None:
        nomeFile = link.split('/')[-1]
    richiesta = requests.get(link)
    if richiesta.status_code == 200:
        with open(nomeFile, 'w') as file:
            file.write(richiesta.text)
            
download('https://tommasoadamo.it/data/oils.json')

In [ ]:
# ALERT: execute this cell to install DOcplex! 
!pip install docplex cplex

In [ ]:
import json
import docplex.mp.model as cplex
import matplotlib.pyplot as plt


with open('oils.json', 'r') as file:
    dati = json.load(file)

n = len(dati['fuels'])
m = len(dati['oils'])
F = list(range(n))
O = list(range(m))
    
R = dati['advert_return']
P = dati['production_cost']
M = dati['production_max']

# informazioni sui combustibili
pF = []
d = []
Llead = []
Loct = []
nameF = []

# informazioni sugli oli
pO = []
C = []
llead = []
loct = []
nameO = []

for o in dati['oils']:
    C.append(o['capacity'])
    pO.append(o['price'])
    llead.append(o['lead'])
    loct.append(o['octane'])
    nameO.append(o['name'])

for f in dati['fuels']:
    pF.append(f['price'])
    d.append(f['demand'])
    Llead.append(f['lead'])
    Loct.append(f['octane'])
    nameF.append(f['name'])

with cplex.Model('miscelazione') as mdl:
    b = mdl.continuous_var_matrix(O, F, name='b')
    a = mdl.continuous_var_list(F, name='a')
    
    mdl.maximize(sum((pF[f] - pO[o] - P) * b[o,f] for f in F for o in O) - sum(a[f] for f in F))
    
    for f in F:
        mdl.add_constraint(sum(b[o,f] for o in O) == d[f] + R*a[f])
        mdl.add_constraint(sum((loct[o]-Loct[f])*b[o,f] for o in O) >= 0)
        mdl.add_constraint(sum((llead[o]-Llead[f])*b[o,f] for o in O) <= 0)
        
    for o in O:
        mdl.add_constraint(sum(b[o,f] for f in F) <= C[o])
    
    mdl.add_constraint(sum(b[o,f] for o in O for f in F) <= M)
    
    mdl.print_information()
    sol = mdl.solve()
    if sol == None:
        print('No solution.')
    else:
        print(sol)
        
        print('Guadagno='+str(sol.objective_value))
        
        ricavo = sum( pF[f]*b[o,f].solution_value  for f in F for o in O)
        print('Ricavo='+str(ricavo))
        
        costo_olio = sum( pO[o]*b[o,f].solution_value for f in F for o in O)
        print('Costo totale olio='+str(costo_olio))
        costo_prod = P*sum(b[o,f].solution_value for f in F for o in O)
        print('Costo totale produzione='+str(costo_prod))
        costo_pubb = sum(a[f].solution_value for f in F)
        print('Costo totale pubblicità='+str(costo_pubb))
        
        for f in F:
            v = [ b[o,f].solution_value for o in O ]
            plt.pie(v, labels=nameO, autopct="%1.1f%%")
            plt.title("Composizione del combustibile: " + nameF[f])
            plt.show()